In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ydata_profiling as pp

## Extract

In [93]:
df= pd.read_csv(r'C:\Users\GTR\Documents\GitHub\cars_sales\ETL_data\vehicles_clean.csv')

## Creating tables

### Time Dimension

In [128]:
D_time = df[['year']].drop_duplicates().sort_values('year').reset_index(drop=True)
D_time['id_time'] = D_time.index
D_time = D_time[['id_time','year']]
D_time.head(2)

,id_time,year
0,0,1900.0
1,1,1905.0


### Location Dimension

In [129]:
D_location = df[['state','region']].drop_duplicates().sort_values('state').reset_index(drop=True)
D_location['id_location'] = D_location.index
D_location = D_location[['id_location','state','region']]
D_location.head(2)

,id_location,state,region
0,0,ak,anchorage / mat-su
1,1,ak,southeast alaska


### Title status Dimension

In [130]:
D_title_status = df[['title_status']].drop_duplicates().sort_values('title_status').reset_index(drop=True)
D_title_status['id_title_status'] = D_title_status.index
D_title_status = D_title_status[['id_title_status','title_status']]
D_title_status.head(2)

,id_title_status,title_status
0,0,clean
1,1,lien


### Paint color Dimension

In [131]:
D_paint_color = df[['paint_color']].drop_duplicates().sort_values('paint_color').reset_index(drop=True)
D_paint_color['id_paint_color'] = D_paint_color.index
D_paint_color = D_paint_color[['id_paint_color','paint_color']]
D_paint_color.head(2)

,id_paint_color,paint_color
0,0,black
1,1,blue


### car Dimension

In [132]:
D_car = df[['manufacturer', 'type']].drop_duplicates().sort_values(['manufacturer', 'type']).reset_index(drop=True)
D_car['id_car'] = D_car.index
D_car = D_car[['id_car','manufacturer', 'type']]
D_car.head(2)

,id_car,manufacturer,type
0,0,acura,SUV
1,1,acura,coupe


### Condition Dimension 

In [133]:
D_condition = df[['condition']].drop_duplicates().sort_values('condition').reset_index(drop=True)
D_condition['id_condition'] = D_condition.index
D_condition = D_condition[['id_condition','condition']]
D_condition.head(2)

,id_condition,condition
0,0,excellent
1,1,fair


### Engine Dimension

In [134]:
D_engine = df[['cylinders', 'fuel', 'transmission']].drop_duplicates().sort_values(['cylinders', 'fuel', 'transmission']).reset_index(drop=True)
D_engine['id_engine'] = D_engine.index
D_engine = D_engine[['id_engine','cylinders', 'fuel', 'transmission']]
D_engine.head(2)

,id_engine,cylinders,fuel,transmission
0,0,10,diesel,automatic
1,1,10,gas,automatic


### Fact table

In [146]:
fact_table = df.merge(D_time, on='year', how='left')\
.merge(D_location, on=['state','region'], how='left')\
.merge(D_title_status, on='title_status', how='left')\
.merge(D_paint_color, on='paint_color', how='left')\
.merge(D_car, on=['manufacturer', 'type'], how='left')\
.merge(D_condition, on='condition', how='left')\
.merge(D_engine, on=['cylinders', 'fuel', 'transmission'], how='left')
fact_table['count'] = 1
fact_table['mean_price'] = fact_table['price'] / fact_table['count']
fact_table['mean_kilometre'] = fact_table['kilometre'] / fact_table['count']
fact_table = fact_table[['id_time','id_location','id_title_status','id_paint_color','id_car','id_condition','id_engine', 'price', 'mean_price', 'kilometre', 'mean_kilometre', 'count']]
fact_table = fact_table.groupby(['id_time','id_location','id_title_status','id_paint_color','id_car','id_condition','id_engine']).agg({'price':'sum','kilometre':'sum','mean_price':'mean','mean_kilometre':'mean','count':'sum'}).reset_index()

In [148]:
fact_table.sort_values('count', ascending=False).head()

,id_time,id_location,id_title_status,id_paint_color,id_car,id_condition,id_engine,price,kilometre,mean_price,mean_kilometre,count
31259,87,366,5,9,339,2,19,42000.0,6.759228e+06,2000.00,321868.00000,21
38437,89,150,0,1,340,0,21,302310.0,4.423316e+06,16795.00,245739.78064,18
33822,88,150,0,9,339,0,19,161010.0,1.992456e+06,8945.00,110692.01454,18
63607,93,283,0,4,101,2,19,71420.0,5.927521e+06,4463.75,370470.06800,16
35348,88,279,0,9,145,2,19,65988.0,2.858188e+06,4124.25,178636.74000,16


## Load 